In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df

,Unnamed: 0,brand,type,model,dimension,color_array,finish,price
0,0,Cristofori,grand,G62L,182.88,['ebony'],['satin'],20999.0
1,1,Fazioli,grand,F308,304.80,['ebony'],"['polish', 'satin']",302000.0
2,2,Fazioli,grand,F308,304.80,"['white', 'other']","['polish', 'satin']",332300.0
3,3,Fazioli,grand,F308,304.80,"['walnut', 'cherry', 'mahogany']","['satin', 'polish']",362500.0
4,4,Fazioli,grand,F308,304.80,"['mahogany', 'macassar']","['satin', 'polish']",392900.0
...,...,...,...,...,...,...,...,...
1992,1992,Brodmann,vertical,CE 118,119.38,['ebony'],['polish'],11890.0
1993,1993,Brodmann,vertical,PE 130,132.08,['ebony'],['polish'],22190.0
1994,1994,Brodmann,vertical,PE 126 Institutaional,124.46,['ebony'],['polish'],17490.0
1995,1995,Sauter,vertical,122 Peter Maly Artes,121.92,"['palisander', 'macassar']",['polish'],63500.0


In [4]:
most_common_finish = df["finish"].mode().iloc[0]
print("Most common finish:", most_common_finish)

Most common finish: ['polish']


In [5]:
def clean_data(finish):
    if finish == '[]':
        return 'polish'
    else:
        return finish
df['finish'] = df['finish'].apply(clean_data)
df.to_csv('data.csv')

In [6]:
df = pd.get_dummies(df, columns=["type"])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1997 non-null   int64  
 1   brand          1997 non-null   object 
 2   model          1997 non-null   object 
 3   dimension      1997 non-null   float64
 4   color_array    1997 non-null   object 
 5   finish         1997 non-null   object 
 6   price          1997 non-null   float64
 7   type_grand     1997 non-null   bool   
 8   type_vertical  1997 non-null   bool   
dtypes: bool(2), float64(2), int64(1), object(4)
memory usage: 113.2+ KB


In [8]:
X = df.drop("price", axis=1)
y = df["price"]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
X_train

,Unnamed: 0,brand,model,dimension,color_array,finish,type_grand,type_vertical
743,743,Schimmel,C 189 Tradition,182.88,['white'],['polish'],True,False
159,159,Yamaha,GC2,152.40,['ebony'],"['polish', 'satin']",True,False
1166,1166,Yamaha,b1 Continental,109.22,['white'],['polish'],False,True
318,318,Bosendorfer,230VC,213.36,['white'],['satin'],True,False
924,924,Blüthner,11 Ambassador,152.40,['walnut'],polish,True,False
...,...,...,...,...,...,...,...,...
835,835,Kayserburg,KA151,152.40,['ebony'],['polish'],True,False
1216,1216,Steingraeber,122 T-SFM,121.92,['ebony'],"['polish', 'satin']",False,True
1653,1653,Blüthner,A,124.46,"['walnut', 'other']",['polish'],False,True
559,559,Steinway & Sons,M,152.40,['pommele'],['polish'],True,False


In [11]:
from sklearn.preprocessing import OrdinalEncoder, MultiLabelBinarizer
import ast
def safe_parse(val):
    try:
        return ast.literal_eval(val)
    except Exception:
        return []
X_train["color_array"] = X_train["color_array"].apply(safe_parse)
X_train["finish"] = X_train["finish"].apply(safe_parse)

color_binarizer = MultiLabelBinarizer()
finish_binarizer = MultiLabelBinarizer()
color_encoded = pd.DataFrame(color_binarizer.fit_transform(X_train["color_array"]),
                             columns=[f"color_{c}" for c in color_binarizer.classes_],
                             index=X_train.index)
finish_encoded = pd.DataFrame(finish_binarizer.fit_transform(X_train["finish"]),
                              columns=[f"finish_{f}" for f in finish_binarizer.classes_],
                              index=X_train.index)

X_train_encoded = pd.concat([X_train.drop(["color_array", "finish"], axis=1),
                             color_encoded, finish_encoded], axis=1)

categorical_cols = ["brand", "model"]
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train_encoded[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])


In [12]:
# from sklearn.ensemble import RandomForestRegressor
# regressor = RandomForestRegressor(
#     n_estimators=200,      # More trees = better generalization
#     max_depth=20,  
#     min_samples_split = 5,# Let trees grow fully
#     min_samples_leaf=2,   # Default, but can tune
#     random_state=0,
#     n_jobs=-1              # Use all CPU cores
# )
# regressor.fit(X_train_encoded, y_train)

In [13]:
from sklearn.ensemble import HistGradientBoostingRegressor

gbm = HistGradientBoostingRegressor(max_iter=300, 
                                    random_state=0,
                                    l2_regularization = 1.0,
                                    learning_rate = 0.2,
                                    max_depth  = 5,
                                    min_samples_leaf = 10
                                    )
gbm.fit(X_train_encoded, y_train)


HistGradientBoostingRegressor(l2_regularization=1.0, learning_rate=0.2,
                              max_depth=5, max_iter=300, min_samples_leaf=10,
                              random_state=0)

In [14]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': [100, 200],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5],
#     'min_samples_leaf': [1, 2]
# }

# grid_search = GridSearchCV(
#     RandomForestRegressor(random_state=0, n_jobs=-1),
#     param_grid,
#     cv=5,
#     scoring='r2',
#     verbose=1
# )

# grid_search.fit(X_train_encoded, y_train)

# print("Best R2:", grid_search.best_score_)
# print("Best Params:", grid_search.best_params_)


In [15]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import HistGradientBoostingRegressor

# param_grid = {
#     'max_iter': [100, 200, 300],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'max_depth': [None, 5, 10],
#     'min_samples_leaf': [20, 10, 5],
#     'l2_regularization': [0.0, 0.1, 1.0]
# }

# grid = GridSearchCV(
#     HistGradientBoostingRegressor(random_state=0),
#     param_grid,
#     scoring='r2',
#     cv=5,
#     n_jobs=-1,
#     verbose=1
# )
# grid.fit(X_train_encoded, y_train)

# print("Best R²:", grid.best_score_)
# print("Best Params:", grid.best_params_)

# best_model = grid.best_estimator_


In [16]:
# Parse array fields
X_test["color_array"] = X_test["color_array"].apply(safe_parse)
X_test["finish"] = X_test["finish"].apply(safe_parse)

# Transform color and finish arrays
color_encoded_test = pd.DataFrame(
    color_binarizer.transform(X_test["color_array"]),
    columns=[f"color_{c}" for c in color_binarizer.classes_],
    index=X_test.index
)

finish_encoded_test = pd.DataFrame(
    finish_binarizer.transform(X_test["finish"]),
    columns=[f"finish_{f}" for f in finish_binarizer.classes_],
    index=X_test.index
)

X_test_encoded = pd.concat(
    [X_test.drop(["color_array", "finish"], axis=1),
     color_encoded_test, finish_encoded_test],
    axis=1
)

X_test_encoded[categorical_cols] = ordinal_encoder.transform(X_test[categorical_cols])


In [17]:
# y_pred = regressor.predict(X_test_encoded)
# np.set_printoptions(precision=2)
# print(np.concatenate((
#     y_pred.reshape(-1, 1),
#     y_test.values.reshape(-1, 1)
# ), axis=1))

In [18]:
y_pred = gbm.predict(X_test_encoded)
np.set_printoptions(precision=0)
print(np.concatenate(
    (y_pred.reshape(-1, 1), np.array(y_test).reshape(-1, 1)),
    axis=1
))

[[186408. 190301.]
 [ 66365.  10752.]
 [ 59101.  53900.]
 [ 50241.  78803.]
 [118987. 107409.]
 [ 17528.  18552.]
 [ 19071.  19130.]
 [128687. 124128.]
 [ 20412.   6695.]
 [ 62193.  36207.]
 [113530. 114652.]
 [176997. 195303.]
 [255427. 256999.]
 [138185.  25956.]
 [118013. 112580.]
 [ 23595.  19460.]
 [ 15834.  14995.]
 [ 34921.  34646.]
 [146427. 147100.]
 [ 40730.  45900.]
 [ 15018.  14290.]
 [146000. 147334.]
 [ 50788.   7900.]
 [ 43163.  56671.]
 [193518. 187507.]
 [ 40762.  11872.]
 [ 30106.  21627.]
 [156940. 173040.]
 [194429. 200000.]
 [ 30628.  27199.]
 [ 26108.  28000.]
 [240064. 238885.]
 [ 84645.  15456.]
 [ 80213.  19190.]
 [ 41880.  41668.]
 [132797. 153452.]
 [ 26504.  27899.]
 [ 32505.  33699.]
 [ 37826.  36178.]
 [ 16359.  12499.]
 [ 31887.  42615.]
 [ 54125.  58322.]
 [ 84282.  78112.]
 [ 53020.  36207.]
 [206695. 198300.]
 [250028. 273800.]
 [132401.  30478.]
 [299265. 349999.]
 [ 46743.  47000.]
 [ 28769.  29937.]
 [ 40889.  43000.]
 [ 12302.  18552.]
 [158759. 21

In [19]:
# from sklearn.metrics import r2_score
# r2_score(y_test, y_pred)

In [20]:
r2 = gbm.score(X_test_encoded, y_test)
print(f"GBM R2 Score: {r2:.2f}")

GBM R2 Score: 0.89


In [21]:
import joblib

preprocessing_objects = {
    "model": gbm,
    "ordinal_encoder": ordinal_encoder,
    "color_binarizer": color_binarizer,
    "finish_binarizer": finish_binarizer,
}

joblib.dump(preprocessing_objects, "model_pipeline.pkl")

['model_pipeline.pkl']